# Spotify Graph Queries

We perform ten queries on our Spotify database relying on ***SPARQLWrapper***.

## Setup

We import all the necessary libraries and we set the URL needed by SPARQLWrapper

In [ ]:
# Imports

import pandas as pd
from pandas import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
SPOTIFY_ONTOLOGY = "https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#"
REPOSITORY_NAME = "eu"

SPARQL_ENDPOINT = "http://localhost:7200/repositories/" + REPOSITORY_NAME

sparql = SPARQLWrapper(SPARQL_ENDPOINT)


## Query utilities
We define a function to perform a SPARQL query

In [ ]:
# Execute a SPARQL query and return a DataFrame
def executeQuery(query, maxRows=None):
    global sparql

    # Set the number of rows and columns to display
    pd.set_option("display.max_rows", maxRows, "display.max_columns", None)

    # Set the query
    sparql.setQuery("""
            prefix so: <https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#>
            prefix countries: <http://eulersharp.sourceforge.net/2003/03swap/countries#>
            prefix xsd: <http://www.w3.org/2001/XMLSchema#>

            {query}
        """.format(query=query))

    # Set the result format
    sparql.setReturnFormat(JSON)

    # Execute the query
    results = sparql.query().convert()

    # If it is an ask
    if "boolean" in results.keys():
        return results["boolean"], "ask"
    
    # If it is a select
    if "results" in results.keys():
        # Create a DataFrame from the returned JSON
        resultDF = json_normalize(results["results"]["bindings"])

        # Remove the datadtypes columns
        resultDF = resultDF[[col for col in resultDF.columns if not col.endswith(
            ".type") and not col.endswith(".datatype")]]

        # Remove .value from column names
        resultDF = resultDF.rename(columns=lambda col: col.replace(".value", ""))

        # Remove the spotify ontology prefix from uris (to get only the ID)
        resultDF = resultDF.applymap(lambda x: str(x).lstrip(SPOTIFY_ONTOLOGY))

        return resultDF, "select"
    
    return None


We define a function to print the result of a query

In [ ]:
def printResult(result, resultType):
    if resultType == "select":
        print("🗃️ [[RESULT]]\n")
        result

    if resultType == "ask":
        print("🎱 [[ANSWER]]\n")
        print("✔️ Yes" if result else "❌ No")


## Queries

#### Query 1: On average how many artist of a specific nationality are in the top 100 of the same country ?


In [ ]:
result, resultType = executeQuery("""
SELECT ?country (xsd:integer(AVG(?num_artists)) AS ?avg_num_artists) WHERE {
	{
		SELECT ?date ?country (COUNT(DISTINCT ?artist) AS ?num_artists) WHERE{
            ?person 		so:isMemberOf ?artist ;
                    		so:hasNationality ?country.
            ?artist 		so:partecipateIn ?track ;
                    		so:name ?name.
            ?track 			so:appearsIn ?appeareance .
            ?appeareance 	so:isPositionedIn ?chart .
            ?chart 			so:isReferredTo ?country ;
                   			so:date ?date .   
    	}
    	GROUP BY ?date ?country
	}
}
GROUP BY ?country
ORDER BY DESC(?avg_num_artists)
""")

printResult(result, resultType)

#### Query 2: Show the 5 most popular young (less than 30 years) italian artist who published in 2018


In [ ]:
result, resultType = executeQuery("""
SELECT DISTINCT ?artistName ?artistPopularity WHERE{
    ?person so:isMemberOf ?artist ;
            so:hasNationality countries:it ;
            so:birthdate ?artistBirthdate .
    ?artist so:partecipateIn ?track ;
            so:name ?artistName ;
    		so:popularity ?artistPopularity .
    ?track so:isPartOf ?album ;
           so:name ?trackName .
	?album so:releaseDate ?date ;
        	so:name ?albumName .
    FILTER ((?date >= "2018-01-01"^^xsd:date && ?date < "2019-01-01"^^xsd:date))
    FILTER (?artistBirthdate >= "1991-01-01"^^xsd:date )
}
""")

printResult(result, resultType)

#### Query 3: Show the 30 genres with most danceable songs relative to songs released in 2019


In [ ]:
result, resultType = executeQuery("""
    SELECT ?genre (AVG(?danceability) as ?avgDanceability) (COUNT(?track) as ?numTrack) WHERE{
    ?artist so:partecipateIn ?track ;
            so:name ?artistName ;
            so:hasGenre ?genre .
    ?track 	so:isPartOf ?album ;
    		so:name ?trackName ;
    		so:danceability ?danceability .
	?album 	so:releaseDate ?date ;
        	so:name ?albumName .
    FILTER ((?date >= "2019-01-01"^^xsd:date && ?date < "2020-01-01"^^xsd:date))
}
GROUP BY ?genre
HAVING (?numTrack > 100)
ORDER BY DESC(?avgDanceability)
LIMIT 30
""")

printResult(result, resultType)

#### Query 4: In 2019 there was more distinct singles in top 100 Argentina or top 100 Italy ?

In [ ]:
result, resultType = executeQuery("""
ASK WHERE{
    {
        SELECT (COUNT (DISTINCT ?track) as ?numSinglesAG) WHERE{
            ?track so:isPartOf ?album ;
             	   so:name ?trackName ;
    			   so:appearsIn ?appearance .
        	?album so:isTypeOf so:single;
                	so:name ?albumName .	
        	?appearance so:isPositionedIn ?chart .
        	?chart so:isReferredTo countries:ar ;
                   so:date ?date .
    		FILTER ((?date >= "2018-01-01"^^xsd:date && ?date < "2019-01-01"^^xsd:date))
		}
    }
    {
        SELECT (COUNT (DISTINCT ?track) as ?numSinglesIT) WHERE{
            ?track so:isPartOf ?album ;
             	   so:name ?trackName ;
    			   so:appearsIn ?appearance .
        	?album so:isTypeOf so:single;
                	so:name ?albumName .	
        	?appearance so:isPositionedIn ?chart .
        	?chart so:isReferredTo countries:it ;
                   so:date ?date .
    		FILTER ((?date >= "2018-01-01"^^xsd:date && ?date < "2019-01-01"^^xsd:date))
		}
    }
    FILTER (?numSinglesAG > ?numSinglesIT)
}
""")

printResult(result, resultType)

#### Query5: How many times artist born after 2000 were present in top 100 Italy in 2020 ?

In [ ]:
result, resultType = executeQuery("""
    select distinct ?trackName ?artistName ?completeName ?birthDate (count(?chartDate) as ?numAppearances) where { 
        ?track a so:Track ;
            so:name ?trackName ;
            so:appearsIn ?appearance .
        ?artist a so:Artist ;
                so:partecipateIn ?track ;
                so:name ?artistName .
        ?artistPerson so:isMemberOf ?artist ;
                    so:name ?realName ;
                    so:surname ?realSurname ;
                    so:birthdate ?birthDate ;
                    so:deathdate ?deathDate .
        ?appearance a so:Appearance ;
                    so:isPositionedIn ?chart.
        ?chart a so:Chart;
            so:isReferredTo countries:it ;
            so:date ?chartDate ;
            so:name ?chartName .
        BIND(CONCAT(?realName, " ", ?realSurname) AS ?completeName) .
        FILTER(?birthDate >= "2000-01-01"^^xsd:date) .
        FILTER(?chartDate > "2020-01-01"^^xsd:date) .
    } group by ?trackName ?artistName ?completeName ?birthDate
""")

printResult(result, resultType)

#### Query 6: Show average energy of tracks through the months of the year


In [ ]:
result, resultType = executeQuery("""
select ?month (avg(?energy) as ?res) where {
    ?track a so:Track ;
           so:appearsIn ?appearance ;
           so:explicit ?explicit ;
           so:danceability ?danceability ;
           so:energy ?energy ;
           so:valence ?valence ;
           so:loudness ?loudness ;
           so:mode ?mode ;
           so:key ?key .
    ?appearance so:isPositionedIn ?chart ;
                so:position ?position .
    ?chart so:date ?date ;
           so:isReferredTo countries:it .
    
    # filter(?position <= "10"^^xsd:int) .
    
    bind(month(?date) as ?month)
    
    # filter(month(?date) <= "3"^^xsd:integer || month(?date) >= "10"^^xsd:integer) .
    # filter(month(?date) < "10"^^xsd:integer || month(?date) > "3"^^xsd:integer) .
} group by ?month order by asc(?month)
""")

printResult(result, resultType)

#### Query 7 : How many albums (at least 2) contains the same track ?

In [ ]:
result, resultType = executeQuery("""
select ?trackNameWithArtists (group_concat( distinct ?albumName; separator=", ") as ?albums) (count(distinct ?album) as ?albumCount) where {
    {
        select ?track ?trackName ?albumType ?album ?albumName (group_concat(?artistName; separator=", ") as ?artists) where {
            ?track a so:Track ;
                   so:name ?trackName ;
                   so:isPartOf ?album .
            ?album so:isTypeOf ?albumType ;
                   so:name ?albumName .
            ?artist so:partecipateIn ?track ;
                    so:name ?artistName .
        } group by ?track ?trackName ?albumType ?album ?albumName
    } .

    bind(concat(?trackName, " (", ?artists, ")") as ?trackNameWithArtists) .
} group by ?trackNameWithArtists
having (count(distinct ?album) > 1)
order by desc(?albumCount)
""")

printResult(result, resultType)

#### Query 8 : Show if there is a correlation between number of explicit songs and available markets

In [ ]:
result, resultType = executeQuery("""
SELECT ?numMarkets (COUNT(?track) AS ?numExplicit) WHERE{
    ?track so:explicit ?explicit
    {  SELECT ?track (COUNT(DISTINCT ?country) AS ?numMarkets) WHERE{
            ?artist so:partecipateIn ?track ;
                    so:name ?artistName .
            ?track 	so:isPartOf ?album ;
                    so:name ?trackName ;
                    so:isAvailableIn ?country ;
                    so:explicit "true"^^xsd:boolean .
        }
        GROUP BY ?track
    }
}
GROUP BY ?numMarkets
ORDER BY ASC(?numMarkets)
""")

printResult(result, resultType)

#### Query 9 : How many songs from Japan or South Korea reaches Top 20 in US, Canada or Mexico?

In [ ]:
result, resultType = executeQuery("""
SELECT ?chartCountry ( COUNT (DISTINCT ?track) AS ?numTracks ) WHERE{
    ?person so:isMemberOf ?artist ;
            so:hasNationality ?originCountry .
    ?artist so:partecipateIn ?track ;
            so:name ?artistName .
    ?track so:name ?trackName ;
           so:appearsIn ?appeareance .
    ?appeareance so:isPositionedIn ?chart ;
                 so:position ?pos.
    ?chart so:isReferredTo ?chartCountry .
    FILTER (?originCountry = countries:jp || ?originCountry = countries:kr)
    FILTER (?chartCountry = countries:us || ?chartCountry = countries:mx || ?chartCountry = countries:ca)
    FILTER (?pos <= 20)
}
GROUP BY ?chartCountry
ORDER BY DESC(?numTracks)
""")

printResult(result, resultType)

#### Query 10: Show the 30 albums with the most high number of songs present in Top 30 at the same time. Show also how many tracks are contained in the album.

In [ ]:
result, resultType = executeQuery("""
    SELECT ?chartName ?chartDate ?albumName ?num_track ?totalTracks WHERE{
    {	?chart  so:name ?chartName ;
             	so:date ?chartDate .
        ?album	so:name ?albumName ;
        		so:totalTracks ?totalTracks .
    }
    {
        SELECT ?chart ?album (COUNT(DISTINCT ?track) AS ?num_track) WHERE{
            ?artist 		so:partecipateIn ?track .               
            ?track 			so:isPartOf ?album ;
                            so:appearsIn ?appereance .
            ?appereance 	so:isPositionedIn ?chart ;
                            so:position ?pos .
            FILTER (?pos < 30)
        }
        GROUP BY ?chart ?album
        ORDER BY DESC(?num_track)
        LIMIT 30
    }    
}
""")

printResult(result, resultType)